<a href="https://colab.research.google.com/github/gamalgazziro/Inteligencia_Artificial_Python/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.2 MB/s eta 0:00:00


In [ ]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination

In [ ]:
red_bayes = BayesianNetwork([("Lluvia","Mantenimiento"),
                              ("Lluvia","Tren"),("Mantenimiento","Tren"),
                               ("Tren","Reunión")])

In [ ]:
#Creamos las distribuciones
tipos_lluvia = ["nula", "ligera", "fuerte"]

lluvia_cpd = TabularCPD(variable = "Lluvia",
                    variable_card = 3,
                    values= [[0.7],[0.2],[0.1]],
                    state_names= {"Lluvia":tipos_lluvia})
print(lluvia)


+----------------+-----+
| Lluvia(nula)   | 0.7 |
+----------------+-----+
| Lluvia(ligera) | 0.2 |
+----------------+-----+
| Lluvia(fuerte) | 0.1 |
+----------------+-----+


In [ ]:
dominio_mantenimiento = ["no","si"]

mantenimiento_cpd = TabularCPD(variable = "Mantenimiento",
                           variable_card= len(dominio_mantenimiento),
                           values= [[0.6,0.8,0.9],[0.4,0.2,0.1]],
                           evidence=["Lluvia"],
                           evidence_card=[3],
                           state_names= {"Mantenimiento": dominio_mantenimiento,"Lluvia":tipos_lluvia})
print(mantenimiento)

+-------------------+--------------+----------------+----------------+
| Lluvia            | Lluvia(nula) | Lluvia(ligera) | Lluvia(fuerte) |
+-------------------+--------------+----------------+----------------+
| Mantenimiento(no) | 0.6          | 0.8            | 0.9            |
+-------------------+--------------+----------------+----------------+
| Mantenimiento(si) | 0.4          | 0.2            | 0.1            |
+-------------------+--------------+----------------+----------------+


In [ ]:
from os import stat
dominio_tren = ["a tiempo", "demorado"]
distribucion_tren = {"a tiempo":[0.9,0.8,0.7,0.6,0.5,0.4], "demorado":[0.1,0.2,0.3,0.4,0.5,0.6]}

tren_cpd = TabularCPD(variable= "Tren",
                  variable_card= len(dominio_tren),
                  values=[distribucion_tren[key] for key in distribucion_tren],
                  evidence= ["Mantenimiento","Lluvia"],
                  evidence_card= [2,3],
                  state_names={"Tren": dominio_tren,
                               "Lluvia":tipos_lluvia,
                               "Mantenimiento": dominio_mantenimiento})
print(tren)

+----------------+-----+-------------------+
| Mantenimiento  | ... | Mantenimiento(si) |
+----------------+-----+-------------------+
| Lluvia         | ... | Lluvia(fuerte)    |
+----------------+-----+-------------------+
| Tren(a tiempo) | ... | 0.4               |
+----------------+-----+-------------------+
| Tren(demorado) | ... | 0.6               |
+----------------+-----+-------------------+


In [ ]:
from numpy.core.fromnumeric import var
dominio_reunion = ["asistir","no asistir"]
distribucion_reunion = {"asistir":[0.9,0.6],"no asistir":[0.1,0.4]}

reunion_cpd = TabularCPD( variable= "Reunión",
                     variable_card= len(dominio_reunion),
                      values=[distribucion_reunion[key] for key in distribucion_reunion],
                      evidence= ["Tren"],
                      evidence_card= [2],
                      state_names={"Reunión": dominio_reunion,"Tren": dominio_tren})
print(reunion)

+---------------------+----------------+----------------+
| Tren                | Tren(a tiempo) | Tren(demorado) |
+---------------------+----------------+----------------+
| Reunión(asistir)    | 0.9            | 0.6            |
+---------------------+----------------+----------------+
| Reunión(no asistir) | 0.1            | 0.4            |
+---------------------+----------------+----------------+


In [ ]:
red_bayes.add_cpds(lluvia_cpd, mantenimiento_cpd, tren_cpd, reunion_cpd)

inferencia = VariableElimination(red_bayes)

consulta_1 = inferencia.query(variables=["Reunión"], evidence= {"Lluvia":"nula", "Mantenimiento":"no"})
print(consulta_1)

+---------------------+----------------+
| Reunión             |   phi(Reunión) |
+=====================+================+
| Reunión(asistir)    |         0.8700 |
+---------------------+----------------+
| Reunión(no asistir) |         0.1300 |
+---------------------+----------------+


In [ ]:
consulta_2 = inferencia.query(variables=["Lluvia"],evidence={"Mantenimiento":"no"})
print(consulta_2)

+----------------+---------------+
| Lluvia         |   phi(Lluvia) |
+================+===============+
| Lluvia(nula)   |        0.6269 |
+----------------+---------------+
| Lluvia(ligera) |        0.2388 |
+----------------+---------------+
| Lluvia(fuerte) |        0.1343 |
+----------------+---------------+


In [ ]:
consulta_3 = inferencia.query(variables=["Lluvia"],evidence={"Tren":"demorado","Reunión":"asistir"})
print(consulta_3)

+----------------+---------------+
| Lluvia         |   phi(Lluvia) |
+================+===============+
| Lluvia(nula)   |        0.6444 |
+----------------+---------------+
| Lluvia(ligera) |        0.2176 |
+----------------+---------------+
| Lluvia(fuerte) |        0.1381 |
+----------------+---------------+


In [ ]:
consulta_4 = inferencia.query(variables=["Lluvia","Mantenimiento"],evidence={"Tren":"demorado","Reunión":"asistir"})
print(consulta_4)

+----------------+-------------------+-----------------------------+
| Lluvia         | Mantenimiento     |   phi(Lluvia,Mantenimiento) |
+================+===================+=============================+
| Lluvia(nula)   | Mantenimiento(no) |                      0.1757 |
+----------------+-------------------+-----------------------------+
| Lluvia(nula)   | Mantenimiento(si) |                      0.4686 |
+----------------+-------------------+-----------------------------+
| Lluvia(ligera) | Mantenimiento(no) |                      0.1339 |
+----------------+-------------------+-----------------------------+
| Lluvia(ligera) | Mantenimiento(si) |                      0.0837 |
+----------------+-------------------+-----------------------------+
| Lluvia(fuerte) | Mantenimiento(no) |                      0.1130 |
+----------------+-------------------+-----------------------------+
| Lluvia(fuerte) | Mantenimiento(si) |                      0.0251 |
+----------------+----------------

In [ ]:
consulta_5 = inferencia.query(variables=["Lluvia"],evidence={"Tren":"demorado","Reunión":"asistir","Mantenimiento":"si"})
print(consulta_5)

+----------------+---------------+
| Lluvia         |   phi(Lluvia) |
+================+===============+
| Lluvia(nula)   |        0.8116 |
+----------------+---------------+
| Lluvia(ligera) |        0.1449 |
+----------------+---------------+
| Lluvia(fuerte) |        0.0435 |
+----------------+---------------+


In [ ]:
consulta_6 = inferencia.query(variables=["Lluvia","Mantenimiento","Tren"],evidence={"Reunión":"no asistir"})
print(consulta_6)

+----------------+-------------------+----------------+----------------------------------+
| Lluvia         | Mantenimiento     | Tren           |   phi(Lluvia,Mantenimiento,Tren) |
+================+===================+================+==================================+
| Lluvia(nula)   | Mantenimiento(no) | Tren(a tiempo) |                           0.2202 |
+----------------+-------------------+----------------+----------------------------------+
| Lluvia(nula)   | Mantenimiento(no) | Tren(demorado) |                           0.0978 |
+----------------+-------------------+----------------+----------------------------------+
| Lluvia(nula)   | Mantenimiento(si) | Tren(a tiempo) |                           0.0978 |
+----------------+-------------------+----------------+----------------------------------+
| Lluvia(nula)   | Mantenimiento(si) | Tren(demorado) |                           0.2609 |
+----------------+-------------------+----------------+----------------------------------+

In [ ]:
consulta_7 = inferencia.query(variables=["Reunión","Mantenimiento"],evidence={"Lluvia":"fuerte","Tren":"demorado"})
print(consulta_7)

+---------------------+-------------------+------------------------------+
| Reunión             | Mantenimiento     |   phi(Reunión,Mantenimiento) |
+=====================+===================+==============================+
| Reunión(asistir)    | Mantenimiento(no) |                       0.4909 |
+---------------------+-------------------+------------------------------+
| Reunión(asistir)    | Mantenimiento(si) |                       0.1091 |
+---------------------+-------------------+------------------------------+
| Reunión(no asistir) | Mantenimiento(no) |                       0.3273 |
+---------------------+-------------------+------------------------------+
| Reunión(no asistir) | Mantenimiento(si) |                       0.0727 |
+---------------------+-------------------+------------------------------+
